In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb


In [5]:
# Load the data
df = pd.read_csv('C:/Users/a0095989/Desktop/important/Thesis/Model_Thesis/imputed_deu_data1.csv')

# Convert Year/Month into a datetime format for each city
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

# Prepare data for splitting by city
train, validate, test = {}, {}, {}

# Loop through each city and split the data
for city in df['City'].unique():
    #print(f"Splitting data for {city}...")
    
    # Filter data for the city
    df_city = df[df['City'] == city].copy()
    df_city.set_index('Date', inplace=True)
    df_city.sort_index(inplace=True)

    # Split data into training, validation, and testing sets
    train[city] = df_city[df_city['Year'] <= 2009]  # Training data up to 2010
    validate[city] = df_city[(df_city['Year'] > 2009) & (df_city['Year'] <= 2011)]  # Validation data (2011)
    test[city] = df_city[(df_city['Year'] == 2012) ]  # Test data (2012)
    
    # You can also print some data samples to verify
    #print(f"Training set size: {train[city].shape}")
    #print(f"Validation set size: {validate[city].shape}")
    #print(f"Testing set size: {test[city].shape}")


Training SARIMA for Aachen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Aachen: 1.3590845908416143
Coefficient of Determination (R²) for Aachen: 0.9252339154734149
Training SARIMA for Augsburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Augsburg: 1.133552072268212
Coefficient of Determination (R²) for Augsburg: 0.9399107256190923
Training SARIMA for Bergisch Gladbach...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bergisch Gladbach: 1.3590845908416143
Coefficient of Determination (R²) for Bergisch Gladbach: 0.9252339154734149
Training SARIMA for Berlin...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Berlin: 1.1668689160265133
Coefficient of Determination (R²) for Berlin: 0.9491959095093339
Training SARIMA for Bielefeld...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bielefeld: 1.1937569936308268
Coefficient of Determination (R²) for Bielefeld: 0.9389793135190547
Training SARIMA for Bochum...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bochum: 1.1937569936308268
Coefficient of Determination (R²) for Bochum: 0.9389793135190547
Training SARIMA for Bonn...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bonn: 1.3590845908416143
Coefficient of Determination (R²) for Bonn: 0.9252339154734149
Training SARIMA for Bottrop...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bottrop: 1.1937569936308268
Coefficient of Determination (R²) for Bottrop: 0.9389793135190547
Training SARIMA for Bremen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bremen: 1.120403994914142
Coefficient of Determination (R²) for Bremen: 0.9519417458195713
Training SARIMA for Bremerhaven...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Bremerhaven: 1.120403994914142
Coefficient of Determination (R²) for Bremerhaven: 0.9519417458195713
Training SARIMA for Brunswick...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Brunswick: 1.1723165988708242
Coefficient of Determination (R²) for Brunswick: 0.9415995931472196
Training SARIMA for Chemnitz...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Chemnitz: 1.2700477791911664
Coefficient of Determination (R²) for Chemnitz: 0.9457338453782329
Training SARIMA for Cologne...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Cologne: 1.3590845908416143
Coefficient of Determination (R²) for Cologne: 0.9252339154734149
Training SARIMA for Cottbus...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Cottbus: 1.1668689160265133
Coefficient of Determination (R²) for Cottbus: 0.9491959095093339
Training SARIMA for Düsseldorf...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Düsseldorf: 1.3590845908416143
Coefficient of Determination (R²) for Düsseldorf: 0.9252339154734149
Training SARIMA for Darmstadt...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Darmstadt: 1.321146344823689
Coefficient of Determination (R²) for Darmstadt: 0.9389546737983254
Training SARIMA for Dortmund...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Dortmund: 1.1937569936308268
Coefficient of Determination (R²) for Dortmund: 0.9389793135190547
Training SARIMA for Dresden...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Dresden: 1.2700477791911664
Coefficient of Determination (R²) for Dresden: 0.9457338453782329
Training SARIMA for Duisburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Duisburg: 1.3590845908416143
Coefficient of Determination (R²) for Duisburg: 0.9252339154734149
Training SARIMA for Erfurt...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Erfurt: 1.2329547568596368
Coefficient of Determination (R²) for Erfurt: 0.9393428038806616
Training SARIMA for Erlangen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Erlangen: 1.2757124932691368
Coefficient of Determination (R²) for Erlangen: 0.9387225837459935
Training SARIMA for Essen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Essen: 1.3590845908416143
Coefficient of Determination (R²) for Essen: 0.9252339154734149
Training SARIMA for Fürth...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Fürth: 1.2757124932691368
Coefficient of Determination (R²) for Fürth: 0.9387225837459935
Training SARIMA for Frankfurt...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Frankfurt: 1.321146344823689
Coefficient of Determination (R²) for Frankfurt: 0.9389546737983254
Training SARIMA for Freiburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Freiburg: 1.184502759200312
Coefficient of Determination (R²) for Freiburg: 0.9289325329987984
Training SARIMA for Göttingen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Göttingen: 1.1723165988708242
Coefficient of Determination (R²) for Göttingen: 0.9415995931472196
Training SARIMA for Gelsenkirchen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Gelsenkirchen: 1.1937569936308268
Coefficient of Determination (R²) for Gelsenkirchen: 0.9389793135190547
Training SARIMA for Gera...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Gera: 1.2329547568596368
Coefficient of Determination (R²) for Gera: 0.9393428038806616
Training SARIMA for Hagen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Hagen: 1.1937569936308268
Coefficient of Determination (R²) for Hagen: 0.9389793135190547
Training SARIMA for Halle...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Halle: 1.2329547568596368
Coefficient of Determination (R²) for Halle: 0.9393428038806616
Training SARIMA for Hamburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Hamburg: 1.120403994914142
Coefficient of Determination (R²) for Hamburg: 0.9519417458195713
Training SARIMA for Hamm...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Hamm: 1.1937569936308268
Coefficient of Determination (R²) for Hamm: 0.9389793135190547
Training SARIMA for Hanover...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Hanover: 1.1723165988708242
Coefficient of Determination (R²) for Hanover: 0.9415995931472196
Training SARIMA for Heidelberg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Heidelberg: 1.2757124932691368
Coefficient of Determination (R²) for Heidelberg: 0.9387225837459935
Training SARIMA for Heilbronn...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Heilbronn: 1.2757124932691368
Coefficient of Determination (R²) for Heilbronn: 0.9387225837459935
Training SARIMA for Herne...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Herne: 1.1937569936308268
Coefficient of Determination (R²) for Herne: 0.9389793135190547
Training SARIMA for Hildesheim...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Hildesheim: 1.1723165988708242
Coefficient of Determination (R²) for Hildesheim: 0.9415995931472196
Training SARIMA for Ingolstadt...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Ingolstadt: 1.1715992377004716
Coefficient of Determination (R²) for Ingolstadt: 0.946892402690316
Training SARIMA for Karlsruhe...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Karlsruhe: 1.3530054211988194
Coefficient of Determination (R²) for Karlsruhe: 0.9293580043045965
Training SARIMA for Kassel...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Kassel: 1.321146344823689
Coefficient of Determination (R²) for Kassel: 0.9389546737983254
Training SARIMA for Kiel...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Kiel: 1.120403994914142
Coefficient of Determination (R²) for Kiel: 0.9519417458195713
Training SARIMA for Koblenz...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Koblenz: 1.3590845908416143
Coefficient of Determination (R²) for Koblenz: 0.9252339154734149
Training SARIMA for Krefeld...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Krefeld: 1.3590845908416143
Coefficient of Determination (R²) for Krefeld: 0.9252339154734149
Training SARIMA for Lübeck...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Lübeck: 1.120403994914142
Coefficient of Determination (R²) for Lübeck: 0.9519417458195713
Training SARIMA for Leipzig...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Leipzig: 1.2329547568596368
Coefficient of Determination (R²) for Leipzig: 0.9393428038806616
Training SARIMA for Leverkusen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Leverkusen: 1.3590845908416143
Coefficient of Determination (R²) for Leverkusen: 0.9252339154734149
Training SARIMA for Ludwigshafen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Ludwigshafen: 1.3530054211988194
Coefficient of Determination (R²) for Ludwigshafen: 0.9293580043045965
Training SARIMA for Mönchengladbach...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Mönchengladbach: 1.3590845908416143
Coefficient of Determination (R²) for Mönchengladbach: 0.9252339154734149
Training SARIMA for Mülheim...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Mülheim: 1.3590845908416143
Coefficient of Determination (R²) for Mülheim: 0.9252339154734149
Training SARIMA for Münster...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Münster: 1.1937569936308268
Coefficient of Determination (R²) for Münster: 0.9389793135190547
Training SARIMA for Magdeburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Magdeburg: 1.1723165988708242
Coefficient of Determination (R²) for Magdeburg: 0.9415995931472196
Training SARIMA for Mainz...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Mainz: 1.321146344823689
Coefficient of Determination (R²) for Mainz: 0.9389546737983254
Training SARIMA for Mannheim...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Mannheim: 1.3530054211988194
Coefficient of Determination (R²) for Mannheim: 0.9293580043045965
Training SARIMA for Moers...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Moers: 1.3590845908416143
Coefficient of Determination (R²) for Moers: 0.9252339154734149
Training SARIMA for Munich...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Munich: 1.133552072268212
Coefficient of Determination (R²) for Munich: 0.9399107256190923
Training SARIMA for Neuss...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Neuss: 1.3590845908416143
Coefficient of Determination (R²) for Neuss: 0.9252339154734149
Training SARIMA for Nuremberg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Nuremberg: 1.1715992377004716
Coefficient of Determination (R²) for Nuremberg: 0.946892402690316
Training SARIMA for Oberhausen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Oberhausen: 1.3590845908416143
Coefficient of Determination (R²) for Oberhausen: 0.9252339154734149
Training SARIMA for Offenbach...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Offenbach: 1.321146344823689
Coefficient of Determination (R²) for Offenbach: 0.9389546737983254
Training SARIMA for Oldenburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Oldenburg: 1.1937569936308268
Coefficient of Determination (R²) for Oldenburg: 0.9389793135190547
Training SARIMA for Osnabrück...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Osnabrück: 1.1937569936308268
Coefficient of Determination (R²) for Osnabrück: 0.9389793135190547
Training SARIMA for Paderborn...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Paderborn: 1.1937569936308268
Coefficient of Determination (R²) for Paderborn: 0.9389793135190547
Training SARIMA for Pforzheim...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Pforzheim: 1.2757124932691368
Coefficient of Determination (R²) for Pforzheim: 0.9387225837459935
Training SARIMA for Potsdam...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Potsdam: 1.1668689160265133
Coefficient of Determination (R²) for Potsdam: 0.9491959095093339
Training SARIMA for Ratisbon...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Ratisbon: 1.1715992377004716
Coefficient of Determination (R²) for Ratisbon: 0.946892402690316
Training SARIMA for Recklinghausen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Recklinghausen: 1.1937569936308268
Coefficient of Determination (R²) for Recklinghausen: 0.9389793135190547
Training SARIMA for Remscheid...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Remscheid: 1.3590845908416143
Coefficient of Determination (R²) for Remscheid: 0.9252339154734149
Training SARIMA for Reutlingen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Reutlingen: 1.2757124932691368
Coefficient of Determination (R²) for Reutlingen: 0.9387225837459935
Training SARIMA for Rostock...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Rostock: 1.0890010171219864
Coefficient of Determination (R²) for Rostock: 0.9591254882043809
Training SARIMA for Saarbrücken...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Saarbrücken: 1.3530054211988194
Coefficient of Determination (R²) for Saarbrücken: 0.9293580043045965
Training SARIMA for Salzgitter...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Salzgitter: 1.1723165988708242
Coefficient of Determination (R²) for Salzgitter: 0.9415995931472196
Training SARIMA for Siegen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Siegen: 1.321146344823689
Coefficient of Determination (R²) for Siegen: 0.9389546737983254
Training SARIMA for Solingen...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Solingen: 1.3590845908416143
Coefficient of Determination (R²) for Solingen: 0.9252339154734149
Training SARIMA for Stuttgart...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Stuttgart: 1.2757124932691368
Coefficient of Determination (R²) for Stuttgart: 0.9387225837459935
Training SARIMA for Trier...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Trier: 1.3530054211988194
Coefficient of Determination (R²) for Trier: 0.9293580043045965
Training SARIMA for Ulm...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Ulm: 1.2757124932691368
Coefficient of Determination (R²) for Ulm: 0.9387225837459935
Training SARIMA for Würzburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Würzburg: 1.2757124932691368
Coefficient of Determination (R²) for Würzburg: 0.9387225837459935
Training SARIMA for Wiesbaden...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Wiesbaden: 1.321146344823689
Coefficient of Determination (R²) for Wiesbaden: 0.9389546737983254
Training SARIMA for Witten...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Witten: 1.1937569936308268
Coefficient of Determination (R²) for Witten: 0.9389793135190547
Training SARIMA for Wolfsburg...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Wolfsburg: 1.1723165988708242
Coefficient of Determination (R²) for Wolfsburg: 0.9415995931472196
Training SARIMA for Wuppertal...


c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\a0095989\Desktop\important\Thesis\Model_Thesis\model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Mean Absolute Error (MAE) for Wuppertal: 1.3590845908416143
Coefficient of Determination (R²) for Wuppertal: 0.9252339154734149
       Aachen  Augsburg  Bergisch Gladbach    Berlin  Bielefeld    Bochum  \
MAE  1.359085  1.133552           1.359085  1.166869   1.193757  1.193757   

         Bonn   Bottrop    Bremen  Bremerhaven  ...    Siegen  Solingen  \
MAE  1.359085  1.193757  1.120404     1.120404  ...  1.321146  1.359085   

     Stuttgart     Trier       Ulm  Würzburg  Wiesbaden    Witten  Wolfsburg  \
MAE   1.275712  1.353005  1.275712  1.275712   1.321146  1.193757   1.172317   

     Wuppertal  
MAE   1.359085  

[1 rows x 81 columns]
      Aachen  Augsburg  Bergisch Gladbach    Berlin  Bielefeld    Bochum  \
R²  0.925234  0.939911           0.925234  0.949196   0.938979  0.938979   

        Bonn   Bottrop    Bremen  Bremerhaven  ...    Siegen  Solingen  \
R²  0.925234  0.938979  0.951942     0.951942  ...  0.938955  0.925234   

    Stuttgart     Trier       Ulm  Würzburg  W

C:\Users\a0095989\AppData\Local\Temp\ipykernel_2116\1380363482.py:44: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  forecast_df.index = pd.date_range(start="2013-01-01", periods=len(forecasts[next(iter(forecasts))]), freq="M")


In [7]:
from sklearn.metrics import mean_absolute_error, r2_score
# Prepare storage for forecasts and evaluation
forecasts = {}
mae_scores = {}
r2_scores = {}

# Loop through each city and train the SARIMA model
for city in train.keys():
    print(f"Training SARIMA for {city}...")
    
    # Get the training data for the city
    df_city_train = train[city]
    
    # Train SARIMA model
    sarima_model = SARIMAX(df_city_train['AverageTemperature'], 
                           order=(1,1,1), 
                           seasonal_order=(1,1,1,12), 
                           enforce_stationarity=False, 
                           enforce_invertibility=False)
    sarima_result = sarima_model.fit()

    # Get the test data for the city
    df_city_test = test[city]
    
    # Forecast for the test set (2012)
    forecast = sarima_result.get_forecast(steps=len(df_city_test))
    forecast_values = forecast.predicted_mean
    
    # Store forecasted values
    forecasts[city] = forecast_values
    
    # Calculate performance metrics (Mean Absolute Error and R²)
    mae = mean_absolute_error(df_city_test['AverageTemperature'], forecast_values)
    r2 = r2_score(df_city_test['AverageTemperature'], forecast_values)
    
    mae_scores[city] = mae
    r2_scores[city] = r2
    
    print(f"Mean Absolute Error (MAE) for {city}: {mae}")
    print(f"Coefficient of Determination (R²) for {city}: {r2}")

# Optionally, store the results in a DataFrame for all cities
forecast_df = pd.DataFrame(forecasts)
forecast_df.index = pd.date_range(start="2013-01-01", periods=len(forecasts[next(iter(forecasts))]), freq="M")

# Save forecast results to CSV
forecast_df.to_csv("sarima_forecast_2013.csv")

# Print MAE and R² scores for all cities
mae_df = pd.DataFrame(mae_scores, index=['MAE'])
r2_df = pd.DataFrame(r2_scores, index=['R²'])
print(mae_df)
print(r2_df)

# Optionally, plot the forecasted values for a sample city (e.g., "Aachen")
plt.figure(figsize=(10, 6))
plt.plot(forecast_df.index, forecast_df['Aachen'], label="Forecast")
plt.plot(df_city_test.index, df_city_test['AverageTemperature'], label="Actual", linestyle='--')
plt.title("SARIMA Forecast vs Actual Temperature (Aachen, 2013)")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.legend()
plt.show()
